In [9]:
import os
import csv
import random
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import BSE

random.seed(42)
np.random.seed(42)

In [10]:
sess_id = "q3_demo_default"

start_time = 0
end_time = 60 * 5

buyers_spec = [
    ('ZIP', 5),
    ('ZIC', 5)
]

sellers_spec = [
    ('ZIP', 5),
    ('SHVR', 5)
]


mrktmakers_spec = [
    ('MMM01', 1, {})
]

traders_spec = {
    'buyers': buyers_spec,
    'sellers': sellers_spec,
    'mrktmakers': mrktmakers_spec
}

sup_range = (100, 200)
dem_range = (100, 200)

supply_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}
]
demand_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}
]

order_schedule = {
    'sup': supply_schedule,
    'dem': demand_schedule,
    'interval': 30,
    'timemode': 'drip-jitter'
}

dumpfile_flags = {
    'dump_strats': False,
    'dump_lobs': False,
    'dump_tape': False,
    'dump_blotters': False,
    'dump_avgbals': True
}

BSE.market_session(
    sess_id=sess_id,
    starttime=start_time,
    endtime=end_time,
    trader_spec=traders_spec,
    order_schedule=order_schedule,
    dumpfile_flags=dumpfile_flags,
    sess_vrbs=True,
)

print(f"Session {sess_id} finished.")
print(f"Check file: {sess_id}_avg_balance.csv")


import BSE
import sys

def mmm01_getorder_forced_patch(self, time, countdown, lob):
    if len(self.orders) < 1 or time < 60:
        order = None
    else:
        quoteprice = self.orders[0].price
        order = BSE.Order(self.tid,
                      self.orders[0].otype,
                      quoteprice,
                      self.orders[0].qty,
                      time, lob['QID'])
        self.lastquote = order
    return order

BSE.TraderMMM01.getorder = mmm01_getorder_forced_patch
print("Must 60s")



MM01 init: n_past_trades=1, bid_percent=0.50000, ask_delta=25


q3_demo_default;  
trader=S08 order=None
trader=B02 order=None
trader=B02 order=None
trader=S05 order=None
trader=B02 order=None
trader=S07 order=None
trader=B04 order=None
trader=B04 order=None
trader=S05 order=None
trader=S07 order=None
trader=B05 order=None
trader=B08 order=None
trader=S06 order=None
trader=S09 order=None
trader=S03 order=None
trader=B06 order=None
trader=S07 order=None
trader=B06 order=None
trader=B09 order=None
trader=S02 order=None
trader=M00 order=None
trader=S01 order=None
trader=S04 order=None
trader=S06 order=None
trader=S04 order=None
trader=B03 order=[B03 Bid P=096 Q=1 T= 1.19 QID:0]
trader=B07 order=None
trader=B07 order=None
trader=B02 order=None
trader=S00 order=None
trader=B00 order=None
trader=S08 order=None
trader=S07 order=None
trader=B07 order=None
trader=S08 order=None
trader=B07 order=None
trader=B00 order=None
trader=B02 order=None
trader=M00 order=None
trader=B01 order=None
trader=B

In [11]:
def get_mmm01_profit_from_avg_balance(sess_id, initial_balance=500.0):
   
    filename = f"{sess_id}_avg_balance.csv"
    if not os.path.exists(filename):
        raise FileNotFoundError(f"{filename} not found. Did you run the session with dump_avgbals=True?")

    with open(filename, 'r') as f:
        rows = list(csv.reader(f))

    last_row = rows[-1]

    mmm_avg_balance = None

    for i in range(4, len(last_row), 4):
        ttype = last_row[i].strip()
        if ttype == 'MMM01':
            try:
                avg_balance_str = last_row[i + 3]
                mmm_avg_balance = float(avg_balance_str)
            except (IndexError, ValueError):
                continue

    if mmm_avg_balance is None:
        raise ValueError("No MMM01 data found in avg_balance file.")

    profit = mmm_avg_balance - initial_balance
    return profit

demo_profit = get_mmm01_profit_from_avg_balance("q3_demo_default")
print("Demo MMM01 profit (approx):", demo_profit)


Demo MMM01 profit (approx): 0.0


In [12]:
def run_single_mmm01_experiment(sess_prefix,
                                run_id,
                                mmm_params,
                                market_type="synthetic",
                                end_time=60 * 5,
                                seed=None):

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    start_time = 0
    sess_id = f"{sess_prefix}_{market_type}_{run_id}"

    if market_type == "synthetic":
        sup_range = (100, 200)
        dem_range = (100, 200)

        supply_schedule = [
            {'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}
        ]
        demand_schedule = [
            {'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}
        ]

    elif market_type == "ibm_offset":
        price_offset_filename = "offset-ibm-1m-170831.csv"
        offsetfn = BSE.offset_from_file(
            price_offset_filename=price_offset_filename,
            time_col=1,
            price_col=5,
            scale_factor=75,
            end_time=end_time
        )

        base_sup = (120, 220)
        base_dem = (80, 180)

        sup_range = (base_sup[0], base_sup[1], offsetfn)
        dem_range = (base_dem[0], base_dem[1], offsetfn)

        supply_schedule = [
            {'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}
        ]
        demand_schedule = [
            {'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}
        ]

    else:
        raise ValueError(f"Unknown market_type: {market_type}")

    order_schedule = {
        'sup': supply_schedule,
        'dem': demand_schedule,
        'interval': 30,
        'timemode': 'drip-jitter'
    }

    buyers_spec = [
        ('ZIP', 5),
        ('ZIC', 5)
    ]
    sellers_spec = [
        ('ZIP', 5),
        ('SHVR', 5)
    ]
    mrktmakers_spec = [
        ('MMM01', 1, mmm_params)
    ]

    traders_spec = {
        'buyers': buyers_spec,
        'sellers': sellers_spec,
        'mrktmakers': mrktmakers_spec
    }

    dumpfile_flags = {
        'dump_strats': False,
        'dump_lobs': False,
        'dump_tape': False,
        'dump_blotters': False,
        'dump_avgbals': True
    }

    BSE.market_session(
        sess_id=sess_id,
        starttime=start_time,
        endtime=end_time,
        trader_spec=traders_spec,
        order_schedule=order_schedule,
        dumpfile_flags=dumpfile_flags,
        sess_vrbs=False
    )

    profit = get_mmm01_profit_from_avg_balance(sess_id)

    return profit

In [51]:
default_params = {
    "n_past_trades": 1,
    "bid_percent": 0.5,
    "ask_delta": 25
}

p_syn = run_single_mmm01_experiment(
    sess_prefix="q3_test",
    run_id="syn",
    mmm_params=default_params,
    market_type="synthetic",
    end_time=60 * 5
)

p_ibm = run_single_mmm01_experiment(
    sess_prefix="q3_test",
    run_id="ibm",
    mmm_params=default_params,
    market_type="ibm_offset",
    end_time=60 * 5
)

print("Synthetic market, MMM01 profit:", p_syn)
print("IBM-offset market, MMM01 profit:", p_ibm)


MM01 init: n_past_trades=1, bid_percent=0.50000, ask_delta=25

MM01 init: n_past_trades=1, bid_percent=0.50000, ask_delta=25

Synthetic market, MMM01 profit: 0.0
IBM-offset market, MMM01 profit: 0.0


In [13]:
n_past_trades_list = [1, 5, 10]
bid_percent_list = [0.4, 0.6, 0.8]
ask_delta_list = [10, 25, 50]

param_grid = list(itertools.product(n_past_trades_list,
                                    bid_percent_list,
                                    ask_delta_list))

print("Number of parameter combinations:", len(param_grid))

N_runs = 5

all_results = []

for (n_past, bid_pct, ask_dlt) in param_grid:
    mmm_params = {
        "n_past_trades": n_past,
        "bid_percent": bid_pct,
        "ask_delta": ask_dlt
    }

    for market_type in ["synthetic", "ibm_offset"]:
        for run in range(N_runs):
            seed = 1000 + run
            profit = run_single_mmm01_experiment(
                sess_prefix="q3",
                run_id=f"{market_type}_{n_past}_{bid_pct}_{ask_dlt}_{run}",
                mmm_params=mmm_params,
                market_type=market_type,
                end_time=60 * 5, 
                                  
                seed=seed
            )

            all_results.append({
                "n_past_trades": n_past,
                "bid_percent": bid_pct,
                "ask_delta": ask_dlt,
                "market_type": market_type,
                "run": run,
                "profit": profit
            })

df_results = pd.DataFrame(all_results)
df_results.to_csv("q3_mmm01_param_sweep_results_stage.csv", index=False)

df_results.head()


Number of parameter combinations: 27
MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=25

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=25

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=25

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=25

MM01 init: n_past_trades=1, bid_percent=0.40000, ask_delta=25

MM01 init: n_past_

,n_past_trades,bid_percent,ask_delta,market_type,run,profit
0,1,0.4,10,synthetic,0,0.0
1,1,0.4,10,synthetic,1,0.0
2,1,0.4,10,synthetic,2,0.0
3,1,0.4,10,synthetic,3,0.0
4,1,0.4,10,synthetic,4,0.0
